## GPU Setup

In [3]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
  raise Exception("Unfortunately this instance does not have a T4, P4 or P100 GPU.Please make sure you've configured Colab to request a GPU instance type.\
  Sometimes Colab allocates a Tesla K80 instead of a T4, P4 or P100. Resetting the instance.If you get a K80 GPU, try Runtime -> Reset all runtimes...")
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


In [4]:
!nvidia-smi

Sat Sep 12 18:42:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import os

## Import Data

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/smile-annotations-final.csv', 
                 names=['id', 'text','category'])
df.set_index('id', inplace=True) 
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


## Preprocessing

In [8]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [9]:
df = df[~df.category.str.contains('\|')]
df.category.value_counts()

nocode          1572
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [10]:
df = df[df['category']!='nocode']
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [11]:
possible_labels = df.category.unique()
possible_labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [12]:
label_dict = {}
for i, label in enumerate(possible_labels):
  label_dict[label] = i
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [13]:
df['label'] = df['category'].replace(label_dict) 
df.head(12)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Train/Validation Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=123,
                                                  stratify=df.label.values)

In [16]:
df['data_type'] = ['not_set']*df.shape[0]

In [17]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [18]:
X_train.shape

(1258,)

In [19]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [20]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Load Tokenizer and Encode Data

In [21]:
!pip install transformers

     |████████████████████████████████| 890kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 13.3MB/s 
     |████████████████████████████████| 890kB 27.6MB/s 
     |████████████████████████████████| 3.0MB 41.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=816180f2c4a563eac095f1f49967748a5e67f2b0da9a7c0ef89b96bcb3411866
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', 
    do_lower_case=True 
)

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use

In [25]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [26]:
len(dataset_train)

1258

In [27]:
len(dataset_val)

223

## Set up BERT Pretrained Model

In [28]:
from transformers import BertForSequenceClassification

In [29]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Create Data Loaders

In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [31]:
batch_size=32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

## Setup Optimizer and Scheduler

In [32]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [33]:
optimizer=AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [34]:
epochs=10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

## Define Performance Metrics

In [35]:
import numpy as np
from sklearn.metrics import f1_score

In [36]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [37]:
def func_accuracy_per_class(preds, labels):
  label_dict_inverse = {v:k for k,v in label_dict.items()}
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

## Create Training Loop

In [38]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [40]:
def evaluate(dataloader_val):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  for batch in dataloader_val:
    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    with torch.no_grad(): #no gradient change here, so no training occurs
      outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader_val) 

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [85]:
epochs = 30

for epoch in tqdm(range(1, epochs+1)):
  model.train()
  
  loss_train_total = 0

  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                      leave=False, disable=False)
  
  for batch in progress_bar:
    model.zero_grad()

    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    outputs = model(**inputs)

    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()

    #prevents vanishing or exploding and helps generalization
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step() 
    scheduler.step() 

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

  #save model every epoch with all layers and weights
  if epoch > 20:
    torch.save(model.state_dict(), f'/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/Models_2/BERT_ft_epoch{epoch}.model')

  tqdm.write(f'\n Epoch {epoch}')

  loss_train_avg = loss_train_total/len(dataloader_train)
  tqdm.write(f'Training Loss: {loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation Loss: {val_loss}')
  tqdm.write(f'F1 score weighted: {val_f1}')



 Epoch 1
Training Loss: 0.22923909053206443
Validation Loss: 0.4163061614547457
F1 score weighted: 0.842669886167644



 Epoch 2
Training Loss: 0.22789723239839077
Validation Loss: 0.41588168484824045
F1 score weighted: 0.842669886167644



 Epoch 3
Training Loss: 0.22819667030125856
Validation Loss: 0.414848376597677
F1 score weighted: 0.842669886167644



 Epoch 4
Training Loss: 0.22066841311752797
Validation Loss: 0.4155685326882771
F1 score weighted: 0.842669886167644



 Epoch 5
Training Loss: 0.22840177547186613
Validation Loss: 0.414565167256764
F1 score weighted: 0.842669886167644


...


 Epoch 26
Training Loss: 0.22119712345302106
Validation Loss: 0.41541673881667
F1 score weighted: 0.842669886167644



 Epoch 27
Training Loss: 0.23566860426217318
Validation Loss: 0.4155941924878529
F1 score weighted: 0.842669886167644



 Epoch 28
Training Loss: 0.22305565029382707
Validation Loss: 0.4151720532349178
F1 score weighted: 0.842669886167644



 Epoch 29
Training Loss: 0.22669858168810605
Validation Loss: 0.4155650522027697
F1 score weighted: 0.842669886167644



 Epoch 30
Training Loss: 0.2295053532347083
Validation Loss: 0.4161872480596815
F1 score weighted: 0.842669886167644



## Load and Evaluate Model

In [41]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
model.to(device)
pass

In [44]:
direc_path = "/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/"
model.load_state_dict(torch.load(os.path.join(direc_path, 'Models_2/BERT_ft_epoch28.model'), map_location=torch.device('cuda')))

<All keys matched successfully>

In [45]:
_, predictions, true_vals = evaluate(dataloader_val)

In [46]:
func_accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 165/171
Class: not-relevant
Accuracy: 23/32
Class: angry
Accuracy: 4/9
Class: disgust
Accuracy: 0/1
Class: sad
Accuracy: 0/5
Class: surprise
Accuracy: 0/5


## Single Input Test

In [51]:
def test_with_input(text, label):
  text = np.array([text])
  label = np.array([label])
  encoded_data_test = tokenizer.batch_encode_plus(
    text,
    add_special_tokens=True,
    return_attention_mask=True,
    max_length=256,
    padding=True,
    return_tensors='pt'
  )

  input_ids_test = encoded_data_test['input_ids']
  attention_masks_test = encoded_data_test['attention_mask']
  labels_test = torch.tensor(label)

  dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

  dataloader_test = DataLoader(
    dataset_test
  )
  _, predictions_test, true_vals_test = evaluate(dataloader_test)
  func_accuracy_per_class(predictions_test, true_vals_test)

In [57]:
### {'angry': 2,'disgust': 3,'happy': 0,'not-relevant': 1,'sad': 4, 'surprise': 5}
text = "What a pleasant feeling from coutinho today on his performance"
label = 0
test_with_input(text, label)

Class: happy
Accuracy: 1/1
